# Basics

Import libraries and functions.

In [2]:
import pandas as pd
import numpy as np
import glob
import os
from pyspark.sql.functions import concat, col, lit, split

Loading database from WDB (https://databank.worldbank.org/home.aspx)

In [5]:
df= pd.read_csv (os.getcwd()+"\Data"+'\WDIData.csv')
df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.936004,17.337896,17.687093,18.140971,18.491344,18.825520,19.272212,19.628009,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.499471,6.680066,6.859110,7.016238,7.180364,7.322294,7.517191,7.651598,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.855399,38.046781,38.326255,38.468426,38.670044,38.722783,38.927016,39.042839,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.794160,32.001027,33.871910,38.880173,40.261358,43.061877,44.270860,45.803485,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,18.663502,17.633986,16.464681,24.531436,25.345111,27.449908,29.641760,30.404935,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93277,Bangladesh,BGD,"Mortality rate, under-5, female (per 1,000 liv...",SH.DYN.MORT.FE,256.4,250.4,244.8,239.8,235.2,231.4,...,39.800000,37.800000,35.800000,34.000000,32.100000,30.300000,28.600000,27.100000,NaN,NaN
93278,Bangladesh,BGD,"Mortality rate, under-5, male (per 1,000 live ...",SH.DYN.MORT.MA,268.2,261.8,255.6,250.2,245.5,241.4,...,44.400000,42.200000,40.200000,38.100000,36.300000,34.400000,32.600000,31.000000,NaN,NaN
93279,Bangladesh,BGD,Multidimensional poverty headcount ratio (% of...,SI.POV.MDIM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93280,Bangladesh,BGD,"Multidimensional poverty headcount ratio, chil...",SI.POV.MDIM.17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To work more comfortably, we remove those columns not useful for us, as country name and indicator code, since with the country code and the indicator name we have the information we need.

In [ ]:
df.drop(columns=["Country Name","Indicator Code"], axis=1, inplace=True)

From the thousand countries we have information about in the worlwide database, we have decided to study 50 of them, groupin gthem by geographical and economical similiarity. By this way, we keep in our df just those countries selected.

In [ ]:
europe_list=['DEU','FRA','SWE','GBR','ESP','HRV','POL','GRC','AUT','NLD']
persian_list=['IRQ','QAT','ARE','SAU','AZE','YEM','YDR','OMN']
naf_list=['DZA','EGY','LBY','ISR','TUR','MAR']
saf_list=['SEN','ZAF','LBR','MOZ','CMR','NGA','GHA']
asia_list=['BGD','IND','VNM','THA','IDN','PHL','KOR']
latam_list=['MEX','BRA','ARG','PER','VEN','COL','CHL','PCZ','CRI']
two_list=['USA','CHN']
country_list=europe_list+persian_list+naf_list+saf_list+asia_list+latam_list+two_list 

In [ ]:
df.loc[df['Country Code'].isin(country_list)]

,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
73695,DZA,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.400000,99.500000,99.500000,99.600000,99.600000,99.600000,99.700000,99.700000,NaN,NaN
73696,DZA,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.300000,98.400000,98.500000,98.600000,98.600000,98.700000,98.700000,98.800000,NaN,NaN
73697,DZA,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,99.900000,NaN,NaN
73698,DZA,Access to electricity (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.002205,99.087013,99.186661,99.350250,99.635490,99.697838,99.500000,99.804131,NaN,NaN
73699,DZA,"Access to electricity, rural (% of rural popul...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,97.288933,97.430611,97.652374,98.126846,99.072006,99.071304,98.681053,99.579903,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381475,YEM,Women who believe a husband is justified in be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381476,YEM,Women who were first married by age 15 (% of w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381477,YEM,Women who were first married by age 18 (% of w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381478,YEM,Women's share of population ages 15+ living wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.100000,38.000000,37.800000,37.700000,37.600000,37.500000,37.300000,37.200000,NaN,NaN


Now we transpose the rows of years into the columns.

In [ ]:
dftras=(df.set_index(["Country Code", "Indicator Name"]).stack().reset_index(name='Value').rename(columns={'level_2':'Date'}))
dftras

,Country Code,Indicator Name,Date,Value
0,AFE,Access to clean fuels and technologies for coo...,2000,11.435351
1,AFE,Access to clean fuels and technologies for coo...,2001,11.805706
2,AFE,Access to clean fuels and technologies for coo...,2002,12.167502
3,AFE,Access to clean fuels and technologies for coo...,2003,12.533893
4,AFE,Access to clean fuels and technologies for coo...,2004,12.916778
...,...,...,...,...
7864969,ZWE,Young people (ages 15-24) newly infected with HIV,2016,14000.000000
7864970,ZWE,Young people (ages 15-24) newly infected with HIV,2017,12000.000000
7864971,ZWE,Young people (ages 15-24) newly infected with HIV,2018,9700.000000
7864972,ZWE,Young people (ages 15-24) newly infected with HIV,2019,9600.000000


In [ ]:
indicators_list=['GDP (current US$)','Literacy rate, adult total (% of people ages 15 and above)', 'Government expenditure on education, total (% of government expenditure)','Net migration','Commercial service exports (current US$)','Exports of goods and services (current US$)','Taxes on international trade (current LCU)','Fertility rate, total (births per woman)','People using at least basic sanitation services (% of population)','Employment in agriculture (% of total employment) (modeled ILO estimate)','Employment in services (% of total employment) (modeled ILO estimate)','Employment in industry (% of total employment) (modeled ILO estimate)','Electricity production from renewable sources, excluding hydroelectric (kWh)','Number of infant deaths','Number of infant deaths','Foreign direct investment, net (BoP, current US$)','Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)','Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)','Research and development expenditure (% of GDP)','Labor force with advanced education (% of total working-age population with advanced education)','Suicide mortality rate (per 100,000 population)','CPIA gender equality rating (1=low to 6=high)','Share of youth not in education, employment or training, total (% of youth population)','Government expenditure on education, total (% of government expenditure)']

df2=dftras.loc[dftras['Indicator Name'].isin(indicators_list)]
pd.set_option('display.max_rows', 10)
df2

,Country Code,Indicator Name,Date,Value
3549,AFE,Commercial service exports (current US$),1976,2.303046e+09
3550,AFE,Commercial service exports (current US$),1977,2.531549e+09
3551,AFE,Commercial service exports (current US$),1978,2.978281e+09
3552,AFE,Commercial service exports (current US$),1979,3.600502e+09
3553,AFE,Commercial service exports (current US$),1980,4.494631e+09
...,...,...,...,...
7863115,ZWE,Total alcohol consumption per capita (liters o...,2000,2.460000e+00
7863116,ZWE,Total alcohol consumption per capita (liters o...,2005,2.770000e+00
7863117,ZWE,Total alcohol consumption per capita (liters o...,2010,3.930000e+00
7863118,ZWE,Total alcohol consumption per capita (liters o...,2015,4.920000e+00


In [ ]:
df2.set_index(["Country Code", "Date"]).pivot(columns="Indicator Name", values="Value").reset_index()

Indicator Name,Country Code,Date,CPIA gender equality rating (1=low to 6=high),Commercial service exports (current US$),"Electricity production from renewable sources, excluding hydroelectric (kWh)",Employment in agriculture (% of total employment) (modeled ILO estimate),Employment in industry (% of total employment) (modeled ILO estimate),Employment in services (% of total employment) (modeled ILO estimate),Exports of goods and services (current US$),"Fertility rate, total (births per woman)",...,"Literacy rate, adult total (% of people ages 15 and above)","Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)",Net migration,Number of infant deaths,People using at least basic sanitation services (% of population),Research and development expenditure (% of GDP),"Share of youth not in education, employment or training, total (% of youth population)","Suicide mortality rate (per 100,000 population)",Taxes on international trade (current LCU),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)"
0,ABW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABW,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.471,...,NaN,NaN,-4323.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABW,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15749,ZWE,2016,4.0,3.486285e+08,NaN,66.879997,7.05,26.070000,4.098132e+09,3.804,...,NaN,133.0,NaN,18753.0,37.529193,NaN,NaN,16.8,272855002.0,NaN
15750,ZWE,2017,4.0,4.289693e+08,NaN,66.480003,6.90,26.629999,3.456997e+09,3.707,...,NaN,NaN,-584288.0,18018.0,36.941673,NaN,NaN,15.0,295088115.0,NaN
15751,ZWE,2018,4.0,4.516801e+08,NaN,66.019997,6.75,27.230000,5.081366e+09,3.615,...,NaN,NaN,NaN,17196.0,36.357160,NaN,NaN,14.0,432769378.0,4.67
15752,ZWE,2019,4.0,5.541892e+08,NaN,66.190002,6.57,27.240000,6.026541e+09,3.531,...,NaN,NaN,NaN,16615.0,35.774337,NaN,44.77,14.1,NaN,NaN


CONVIERTE A INTEGER PERO NO FILTRA

Our time range covers from 1960 to 2021. However, the record is not uniform and complete for all areas and indicators. We can appreaciate that specially in the first years of the last century, so many data is missing, so it makes no sense to study it. Besides, for the year 2021 many data is also lacking. Therefore, we would delimit our study between 1990 and 2020.

In [9]:
pd.to_numeric(df2['Date'])

3549       1976
3550       1977
3551       1978
3552       1979
3553       1980
           ... 
7863115    2000
7863116    2005
7863117    2010
7863118    2015
7863119    2018
Name: Date, Length: 128579, dtype: int64

In [4]:
df2.dtypes

NameError: name 'df2' is not defined

In [10]:
df2['Date'>1990]

TypeError: '>' not supported between instances of 'str' and 'int'

NEXT STEP: NORMALIZATION

- Group countries by group list name.
- Nan values: replace by 0, by the mean or eliminating.
- Remove outliers.
- For the main variable to compare (GDP): analyse distribution: if not normal, make logarithmic.

In [53]:
selected['GDP (current US$)'].describe()

KeyError: 'GDP (current US$)'

In [ ]:
sns.displot(selected['GDP (current US$)'])

In [ ]:
selected.mean()
selected.fillna(selected.mean())